Log on wandb: https://wandb.ai/mlteamaau/Twitter%20-%20sentimental%20analysis?nw=nwuserqfaithencz

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv
/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv


In [1]:
from tqdm import tqdm
import re
import nltk
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from collections import Counter
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import wandb
wandb.login()

tqdm.pandas()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [20]:
# Load the dataset
data = pd.read_csv('/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv', names=['id','entity','target','text'],index_col=False)
data = data.drop(columns = ['id','entity'])
# Display the first few rows of the dataframe
data.dropna(inplace=True)
data.head()


,target,text
0,Positive,im getting on borderlands and i will murder yo...
1,Positive,I am coming to the borders and I will kill you...
2,Positive,im getting on borderlands and i will kill you ...
3,Positive,im coming on borderlands and i will murder you...
4,Positive,im getting on borderlands 2 and i will murder ...


In [6]:
# Load the dataset
test_data = pd.read_csv('/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv', names=['id','entity','target','text'],index_col=False)
test_data = test_data.drop(columns = ['id','entity'])
# Display the first few rows of the dataframe
test_data.dropna(inplace=True)
test_data.head()

,target,text
0,Irrelevant,I mentioned on Facebook that I was struggling ...
1,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,Negative,@Microsoft Why do I pay for WORD when it funct...
3,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,Neutral,Now the President is slapping Americans in the...


In [ ]:
!python -m nltk.downloader all
import subprocess
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

In [21]:
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(nltk.corpus.stopwords.words('english'))
def clean_text(text):
    lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)     # Remove mentions
    text = re.sub(r'#\w+', '', text)     # Remove hashtags
    text = re.sub(r'[^A-Za-z\s]', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    text = nltk.word_tokenize(text)  # Tokenize
    text = [word for word in text if word not in stop_words]
    text = [lemmatizer.lemmatize(t) for t in text]
    text = [word for word in text if word not in stop_words]
    return ' '.join(text)

data['cleaned_text'] = data['text'].progress_apply(clean_text)
test_data['cleaned_text'] = test_data['text'].progress_apply(clean_text)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


100%|██████████| 1000/1000 [00:00<00:00, 3124.22it/s]


In [22]:
data['tokens'] = data['cleaned_text'].progress_apply(lambda x: x.split())
test_data['tokens'] = test_data['cleaned_text'].progress_apply(lambda x: x.split())


100%|██████████| 1000/1000 [00:00<00:00, 265748.21it/s]


In [24]:
# Build vocabulary
all_tokens = [token for sublist in data['tokens'] for token in sublist]
vocab = Counter(all_tokens)
vocab = {word: i+2  for i, (word, feq) in enumerate(vocab.most_common()) if feq > 2}
vocab['<PAD>'] = 0
vocab['<UNK>'] = 1


data['token_ids'] = data['tokens'].apply(lambda x: [vocab.get(word, vocab['<UNK>']) for word in x])
test_data['token_ids'] = test_data['tokens'].apply(lambda x: [vocab.get(word, vocab['<UNK>']) for word in x])

In [25]:
# Define pad features function
def pad_features(reviews, pad_id, seq_length=128):
    # features = np.zeros((len(reviews), seq_length), dtype=int)
    features = np.full((len(reviews), seq_length), pad_id, dtype=int)

    for i, row in enumerate(reviews):
        # if seq_length < len(row) then review will be trimmed
        features[i, max(0,seq_length - len(row)):] = np.array(row)[:min(seq_length, len(row))]

    return features

def get_data(batch_size,seq_len,data=data,test_data=test_data):
    
    pad_id = vocab['<PAD>']
    features = pad_features(data['token_ids'], pad_id, seq_len)
    features_test = pad_features(test_data['token_ids'], pad_id, seq_len)
    
    label_dict = {'Positive': 0, 'Negative': 3, 'Neutral': 1, 'Irrelevant': 2}
    labels = data['target'].map(label_dict).values
    labels_test = test_data['target'].map(label_dict).values
    
    X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.3, random_state=42)
    
    train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
    valid_data = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val))
    test_data = TensorDataset(torch.from_numpy(features_test), torch.from_numpy(labels_test))
    
    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
    valid_loader = DataLoader(valid_data, batch_size=batch_size)
    test_loader = DataLoader(test_data, batch_size=batch_size)
    
    return train_loader, valid_loader, test_loader

In [33]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, pad_idx,dropout=None, bidirectional=False):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.bidirectional = bidirectional
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True,bidirectional=bidirectional)
        if self.bidirectional == False:
            self.fc = nn.Linear(hidden_dim, output_dim)
        else:
            self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = dropout
        if dropout is not None:
            self.drop = nn.Dropout(dropout)
        
    def forward(self, text):
        if self.bidirectional:
            h0 = torch.zeros(self.num_layers*2, len(text), self.hidden_dim,device=device) 
        else:
            h0 = torch.zeros(self.num_layers, len(text), self.hidden_dim,device=device)
        embedded = self.embedding(text)
        embedded = embedded.to(torch.float32)
        Y, _ = self.rnn(embedded,h0)
        output = self.fc(Y)
        if self.dropout is not None:
            output = self.drop(output)
        return output[:,-1,:]
    

In [28]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, pad_idx,dropout=None, bidirectional=False):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.bidirectional = bidirectional
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True,bidirectional=bidirectional)
        if self.bidirectional == False:
            self.fc = nn.Linear(hidden_dim, output_dim)
        else:
            self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = dropout
        if dropout is not None:
            self.drop = nn.Dropout(dropout)
        
    def forward(self,text):
        if self.bidirectional:
            c0 = torch.zeros(self.num_layers*2, len(text), self.hidden_dim,device=device)
            h0 = torch.zeros(self.num_layers*2, len(text), self.hidden_dim,device=device)
        else:
            c0 = torch.zeros(self.num_layers, len(text), self.hidden_dim,device=device)
            h0 = torch.zeros(self.num_layers, len(text), self.hidden_dim,device=device)
        embedded = self.embedding(text)
        embedded = embedded.to(torch.float32)
        Y, _ = self.rnn(embedded,(h0,c0))
        output = self.fc(Y)
        if self.dropout is not None:
            output = self.drop(output)
        return output[:,-1,:]
    

In [35]:
sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'accuracy',
    'goal': 'maximize',
    'target': 1.0
    }
sweep_config['metric'] = metric
sweep_config['name'] = 'RNN tuning Twitter dataset'
parameters_dict = {
    'num_hidden': {
        'values': [1, 32, 64]
        },
    'embedded_size': {
        'values': [128, 256, 512]
        },
    'num_layers': {
        'values': [1, 2, 4,8]
        },
    'bidirectional': {
        'values': [False,True]
        },
    'dropout': {
          'values': [0, 0.3, 0.4, 0.5]
        },
    'learning_rate':{
        'values':[0.001, 0.005, 0.01]
    },
    'batch_size':{
        'values':[32,64,128]
    },
    'seq_len':{
        'values':[32,64,128]
    }
    }

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'epochs': {
        'value': 10}
    })



In [36]:
sweep_id = wandb.sweep(sweep_config, project="Twitter - sentimental analysis")

Create sweep with ID: n7hths3o
Sweep URL: https://wandb.ai/mlteamaau/Twitter%20-%20sentimental%20analysis/sweeps/n7hths3o


In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def init_model(embedded_size,num_hidden,num_layers,dropout,bidirectional):
    return RNN(len(vocab),embedded_size, num_hidden, 4, num_layers, vocab['<PAD>'],dropout,bidirectional)

def init_optimizer(model,lr):
    return optim.Adam(model.parameters(),lr=lr)

def log(metrics):
    wandb.log(metrics)

def evaluate(model,iterator,criterion):
    epoch_loss = 0
    epoch_accuracy = 0 
    n = 0

    model.eval()

    with torch.no_grad():
        for texts, labels in iterator:
            texts, labels = texts.to(device), labels.to(device)
            predictions = model(texts)
            loss = criterion(predictions, labels)
            THP = torch.argmax(predictions, dim = 1)
            epoch_accuracy += (THP == labels).float().mean().item()
            n += 1
            epoch_loss += loss.item()

    return epoch_loss / len(iterator), epoch_accuracy/n

def train(model,iterator,optimizer,criterion):
    epoch_loss = 0
    epoch_accuracy = 0 
    n = 0

    model.train()

    for texts, labels in iterator:
        texts, labels = texts.to(device), labels.to(device)
        optimizer.zero_grad()
        predictions = model(texts)
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()
        THP = torch.argmax(predictions, dim = 1)
        epoch_accuracy += (THP == labels).float().mean().item()
        n += 1
        epoch_loss += loss.item()

    return epoch_loss / len(iterator), epoch_accuracy/n

def fit(config=None):
    with wandb.init(config=config):
        config = wandb.config
        
        train_loader, valid_loader, test_loader = get_data(config.batch_size,config.seq_len)
        model = init_model(config.embedded_size,config.num_hidden,config.num_layers,config.dropout,config.bidirectional).to(device)
        optimizer = init_optimizer(model,config.learning_rate)
        criterion = nn.CrossEntropyLoss().to(device)

        for epoch in tqdm(range(config.epochs)):

            train_loss,train_accuracy = train(model, train_loader, optimizer, criterion)
            valid_loss,valid_accuracy = evaluate(model, valid_loader, criterion)
            loss,accuracy = evaluate(model, test_loader, criterion)
            wandb.log({
                'training loss':train_loss,
                'training accuracy': train_accuracy,
                'valid loss':valid_loss,
                'valid accuracy': valid_accuracy,
                'loss': loss,
                'accuracy': accuracy
            })



In [ ]:
wandb.agent(sweep_id, fit)

wandb: Agent Starting Run: h60i46gj with config:
wandb: 	batch_size: 128
wandb: 	bidirectional: False
wandb: 	dropout: 0.5
wandb: 	embedded_size: 256
wandb: 	epochs: 10
wandb: 	learning_rate: 0.01
wandb: 	num_hidden: 64
wandb: 	num_layers: 8
wandb: 	seq_len: 64


100%|██████████| 10/10 [00:26<00:00,  2.66s/it]


accuracy,█▁██▁▁▆▁▁█
loss,▁▂▃▃█▇▃▂▄▆
training accuracy,▆▆▁▅▂▃▂▆█▃
training loss,▅▁▅▃▃▄█▃▃▆
valid accuracy,▁█▁▁██▅██▁
valid loss,▃▁▆▅▆▃▆▃▄█
accuracy,0.28463
loss,1.38889
training accuracy,0.2742
training loss,1.4003
valid accuracy,0.24413


wandb: Agent Starting Run: 4tbhpxpr with config:
wandb: 	batch_size: 64
wandb: 	bidirectional: True
wandb: 	dropout: 0.3
wandb: 	embedded_size: 256
wandb: 	epochs: 10
wandb: 	learning_rate: 0.005
wandb: 	num_hidden: 1
wandb: 	num_layers: 4
wandb: 	seq_len: 128


100%|██████████| 10/10 [00:46<00:00,  4.70s/it]


...

wandb: Agent Starting Run: ey9iq7bo with config:
wandb: 	batch_size: 128
wandb: 	bidirectional: True
wandb: 	dropout: 0.3
wandb: 	embedded_size: 128
wandb: 	epochs: 10
wandb: 	learning_rate: 0.005
wandb: 	num_hidden: 1
wandb: 	num_layers: 8
wandb: 	seq_len: 32


100%|██████████| 10/10 [00:27<00:00,  2.77s/it]


accuracy,▁▆▅▄▆▆▆▆▇█
loss,█▅▅▅▃▃▃▃▂▁
training accuracy,▁▄▆▆▆▇▇▇██
training loss,█▆▄▄▃▃▂▂▁▁
valid accuracy,▁▃▅▅▆▆▇▇██
valid loss,█▆▅▅▃▃▃▃▂▁
accuracy,0.43059
loss,1.249
training accuracy,0.38061
training loss,1.29412
valid accuracy,0.3917


wandb: Agent Starting Run: b5bu74ca with config:
wandb: 	batch_size: 128
wandb: 	bidirectional: False
wandb: 	dropout: 0.5
wandb: 	embedded_size: 256
wandb: 	epochs: 10
wandb: 	learning_rate: 0.001
wandb: 	num_hidden: 1
wandb: 	num_layers: 8
wandb: 	seq_len: 128


 50%|█████     | 5/10 [00:13<00:13,  2.71s/it]